<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Run_models_on_new_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

Clone repositories:

In [1]:
!git clone https://github.com/thtrieu/darkflow.git

Cloning into 'darkflow'...
remote: Enumerating objects: 2713, done.
remote: Total 2713 (delta 0), reused 0 (delta 0), pack-reused 2713
Receiving objects: 100% (2713/2713), 32.98 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (1762/1762), done.


In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

In [0]:
!git clone --quiet https://github.com/aarcosg/traffic-sign-detection.git

# Imports

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
%tensorflow_version 1.x 
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob


In [5]:
!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml
!pip install --user jupyter
!pip install --user matplotlib
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

Selecting previously unselected package python-bs4.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [0]:
import sys
sys.path.append('/content/models/research/slim')

In [7]:
import os
os.chdir('models/research')
!protoc object_detection/protos/*.proto --python_out=.
!pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017522 sha256=5a0e7133f1d8632db8cf6c9d8256fe2ea4dc22a1fe5589ed91b48cea4749e207
  Stored in directory: /tmp/pip-ephem-wheel-cache-7oixk94s/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


In [8]:
print(os.path.abspath(''))
os.chdir('../../')
print(os.path.abspath(''))

/content/models/research
/content


In [9]:
%matplotlib inline
import glob
ROOT_DIR = os.path.abspath('')
PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER = os.path.join(ROOT_DIR, 'models/research/object_detection')
print(list(glob.iglob(PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER + '/*')))
print(PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER)
PATH_TO_DARKFLOW_FOLDER = 'content/darkflow'
# Append your Tensorflow object detection and darkflow directories to your path
sys.path.append(PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER) # ~/tensorflow/models/research/object_detection
sys.path.append(PATH_TO_DARKFLOW_FOLDER) # ~/darkflow
from utils import label_map_util
from utils import visualization_utils as vis_util

['/content/models/research/object_detection/dockerfiles', '/content/models/research/object_detection/object_detection_tutorial.ipynb', '/content/models/research/object_detection/g3doc', '/content/models/research/object_detection/model_lib.py', '/content/models/research/object_detection/dataset_tools', '/content/models/research/object_detection/model_lib_v2_test.py', '/content/models/research/object_detection/exporter_test.py', '/content/models/research/object_detection/tpu_exporters', '/content/models/research/object_detection/predictors', '/content/models/research/object_detection/inputs.py', '/content/models/research/object_detection/export_inference_graph.py', '/content/models/research/object_detection/model_lib_test.py', '/content/models/research/object_detection/test_images', '/content/models/research/object_detection/utils', '/content/models/research/object_detection/inputs_test.py', '/content/models/research/object_detection/data_decoders', '/content/models/research/object_detec

# Tensorflow Object Detection API

In [10]:
# Download model
!pip install gdown

In [11]:
# Download faster_rcnn_inception_resnet_v2_atrous.zip
!gdown 'https://drive.google.com/uc?id=12vLvA9wyJ9lRuDl9H9Tls0z5jsX0I0Da'


Downloading...
From: https://drive.google.com/uc?id=12vLvA9wyJ9lRuDl9H9Tls0z5jsX0I0Da
To: /content/faster_rcnn_inception_resnet_v2_atrous.zip
222MB [00:02, 102MB/s]


In [12]:
!unzip /content/faster_rcnn_inception_resnet_v2_atrous.zip -d /content/faster_rcnn_inception_resnet_v2_atrous/
!rm -r faster_rcnn_inception_resnet_v2_atrous.zip

Archive:  /content/faster_rcnn_inception_resnet_v2_atrous.zip
   creating: /content/faster_rcnn_inception_resnet_v2_atrous/inference_graph/
  inflating: /content/faster_rcnn_inception_resnet_v2_atrous/inference_graph/frozen_inference_graph.pb  
  inflating: /content/faster_rcnn_inception_resnet_v2_atrous/faster_rcnn_inception_resnet_v2_atrous_gtsdb3.config  
  inflating: /content/faster_rcnn_inception_resnet_v2_atrous/detections_output_result.pkl  


## Model preparation 

In [0]:
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
# MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [0]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join(MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/traffic-sign-detection/scripts/gtsdb3_label_map.pbtxt')

NUM_CLASSES = 43

## Load a (frozen) Tensorflow model into memory

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [16]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(label_map)

item {
  name: "prohibitory"
  id: 1
}
item {
  name: "mandatory"
  id: 2
}
item {
  name: "danger"
  id: 3
}



## Helper code

In [0]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

## Detection

In [33]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'images'
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

['images/ZzZvgZzpEwloaJPQzkcHZw.jpg', 'images/KYql_ZQlJeJj192TPN3kvQ.jpg']


In [0]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(image_np)

## Model preparation 

## Helper code

In [0]:
def plot_rectangle(bbox, ax, class_name, edgecolor, confidence=None):
    xmin = bbox[0]
    ymin = bbox[1]
    xmax = bbox[2]
    ymax = bbox[3]
    left = xmin
    right = xmax
    top = ymin
    bot = ymax
    ax.add_patch(
        plt.Rectangle((left, top),
                      right-left,
                      bot-top, fill=False,
                      edgecolor=edgecolor, linewidth=3.5)
        )
    label = '{:s}'.format(class_name)
    label_pos_y = top-10
    if confidence:
        label += ' {0:.2f}'.format(confidence)
        label_pos_y = bot+20
    ax.text(left, label_pos_y,label,
            bbox=dict(facecolor=edgecolor, alpha=0.5),
            fontsize=14, color='white')

In [0]:
def get_label_id(label_name):
    for category in categories:
        if category['name'] == label_name:
            return category['id']

## Detection

In [31]:
min_score_thresh = 0.5
for image_path in TEST_IMAGE_PATHS:
    fig, ax = plt.subplots(figsize=(20, 20))
    image = Image.open(image_path)
    image_name = os.path.basename(image_path)
    width, height = image.size
    ax.imshow(image)
    
    image_np = load_image_into_numpy_array(image)
    image_np = image_np[:,:,::-1] # rgb -> bgr
    pred_results = tfnet.return_predict(image_np)

    for idx, det in enumerate(pred_results):
        score = det['confidence']
        if score > min_score_thresh:
            bbox = det['topleft']['x'], det['topleft']['y'], det['bottomright']['x'], det['bottomright']['y']
            label = get_label_id(det['label'])
            plot_rectangle(bbox,ax,category_index[label]['name'],'red', score)
    plt.draw()
    fig.tight_layout()
    plt.axis('off')
    plt.show()

NameError: ignored